# Techniques for Speeding Up Model Training

## How to scale | Summary

When we train DL models there are a two aspects we want to optimize at the same time:

- Data throughput / training time
- Model performance

We have seen that each method described in the doc changes the memory usage and throughput. In general we want to **maximize the throughput** (samples/second) to minimize the training cost. This is generally achieved by utilizing the GPU as much as possible and thus filling GPU memory to its limit.

For example, as mentioned earlier, we only employ gradient accumulation when we want to use a batch size beyond the size of the GPU memory. If the desired batch size fits into memory then there is no reason to apply gradient accumulation which will only slow down training.

The second objective is **model performance**. Just because we can does not mean we should use a large batch size. As part of hyperparameter tuning you should determine which batch size yields the best result and then optimize the throughput accordingly.

**Hardware choice**. Sometimes, even when applying all the above tweaks the throughput on a given GPU might still not be good enough. One *easy solution* is to change the type of GPU. For example switching from let's say a K80 (which you typically get on Google Colab) to a fancier GPU such as the V100 or A100. Although they are more expensive they are usually more cost effective than cheaper GPUs due to their larger memory and faster architecture. For some applications, such as pretraining, this might still not be fast enough. In this case you want to scale your experiment to several GPUs.

## Anatomy of Model's Memory

The components on GPU memory are the following:

- model weights
    - 4 bytes * number of parameters for fp32 training
    - 6 bytes * number of parameters for mixed precision training

- optimizer states
    - 8 bytes * number of parameters for normal AdamW (maintains 2 states)
    - 2 bytes * number of parameters for 8-bit AdamW optimizers like bitsandbytes
    - 4 bytes * number of parameters for optimizers like SGD (maintains only 1 state)

- gradients
    - 4 bytes * number of parameters for either fp32 or mixed precision training

- forward activations saved for gradient computation
    - size depends on many factors, the key ones being sequence length, hidden size and batch size.

- temporary buffers
    - all kinds of temporary variables which get released once the calculation is done

- functionality-specific memory
    - Then your software could have special memory needs. For example, when generating text using beam search, the software needs to maintain multiple copies of inputs and outputs.

In [ ]:
import torch

## 1) Accelerated Model Training via Mixed-Precision Training

`Mixed-precision training` involves using a combination of different numerical precisions (typically float32 and float16 or bfloat16) during model training to improve computational efficiency and speed.

Traditional training methods tend to use 32-bit floating-point numbers (float32) to represent weights, biases, activations, and gradients for neural networks. However, this can be computationally expensive and memory-intensive, particularly for large models and data sets.

To address this, `mixed-precision training` employs lower-precision formats, namely 16-bit floating-point numbers (float16) and Brain Floating Point (bfloat16), in parts of the training process where higher precision is not critical.

`Mixed precision training`: switching between float32 and float16 representations during training.

**Impact**: Fewer bits to represent a number --> requrie less memory & faster to compute numbers with lower bit representation.

The balance between speed, memory usage, and precision makes mixed-precision training an increasingly popular approach for training large-scale machine learning models.

Reference:
https://lightning.ai/pages/community/tutorial/accelerating-large-language-models-with-mixed-precision-techniques/

*Example*:
DistilBert with float32 - 21 mins with acc 92%
DistilBert with mixed precision training - 7 mins (x3 faster) with acc 92% (identical).

The only thing to change - add precision argument = '16-mixed'.

In [ ]:
# What are min and max values for float32 and float16
# use torch.finfo to find out.
# +/- 3.4 * 10^38 and +/- 65,504

torch.finfo()

finfo(resolution=1e-06, min=-3.40282e+38, max=3.40282e+38, eps=1.19209e-07, smallest_normal=1.17549e-38, tiny=1.17549e-38, dtype=float32)

In [ ]:
torch.set_printoptions(precision=35)

In [ ]:
torch.tensor(1/3, dtype=torch.float32)

tensor(0.33333334326744079589843750000000000)

In [ ]:
torch.tensor(1/3, dtype=torch.float16)
# lower precision

tensor(0.33325195312500000000000000000000000, dtype=torch.float16)

**Lower Precision**. Since in NN we have a lot of stochastic elements and a lot of randommess anyways, we do not necessary need a very high-precision.

But might suffer from **Overflow and Underflow** - representing too large / too small numbers - unexpected results.

In [ ]:
torch.tensor(10**6, dtype=torch.float32)

tensor(1000000.)

In [ ]:
torch.tensor(10**6, dtype=torch.float16)

tensor(inf, dtype=torch.float16)

Automatic `mixed precision training` - network automatically switching between 32-bit and 16-bit precision representations. --> Save computation time without suffering from any accuracy loss.

Under the hood:
1) Convert weights FP32 --> FP16 weights
2) FP16 weights --> FP16 gradients
3) FP16 gradients --> FP32 gradients
4) Optimizer (update weights)
5) Repeat

### Quantization vs Mixed precision training

**Mixed precision training** - switch between 32bit representations and 16bit representations. Speedup *model training*.

**Quantization** - convert 32 bit floats -> 8 bit integer representations to speed up *inference of models*.

## 2) Multi-GPU Training Strategies

GPU vs CPU:
- GPUs have a much higher number of cores compared to CPUs (e.g. 10240 vs 16) -> able to carry out computations (like matrix multiplications) faster.
- Memory Bandwidth greater (912 Gb/s vs 45 Gb/s)
- floating-point calculations: 742 GFLOPS vs 34 TFLOPS.

GPUs are well-suited for parallel computations, making them ideal for training machine learning models.

`GPU training on multiple GPUs` - offers benefits and strategies for large-scale machine learning tasks.

---
Broad categories of parallelism (`Multi-GPU training strategy`) for multi-GPUs training:

- data parallelism
    - increase the training throughput
    - split batch to train model(s) on more data in parallel
    - e.g. create 2 copies of model, put 1 on cude:0, put another on cuda:1, train both models in parallel, average the gradients when we update the models --> go through data faster.
    - data parallelism involves distributing different subsets of the training data across multiple GPUs and then aggregating the gradients for the model update

- model parallelism
    - used in the context of limited GPU memory
    - put different layers on different GPUs to work around GPU memory limitations
    - model parallelism splits the model itself across GPUs, where each GPU computes a part of the forward and backward pass.

- tensor parallelism
    - related to model parallelism, but split layers horizontally instead of vertically.
    - deals with limited GPU memory.
    - the computation of each layer is split across multiple GPUs.
    - more recent approach that splits the model's tensors across multiple GPUs to handle extremely large models that don't fit into a single GPU memory.

- pipeline parallelism
    - mix between data and model parallelism where model is split across different blocks.
    - optimized so that GPUs can work better in parallel.

- sequence parallelism
    - Specifically developed for transformers models.
    - Splits the input sequence into smaller chunks that can be distributed across GPUs to work around memory limitations.


**Impact**. These techniques allow for the optimization of computational resources, speed up the training process, and enable the handling of larger models and datasets, thereby making multi-GPU training a key aspect of modern machine learning infrastructure.

Default choice - **DistributedDataParallel** (multiple GPUs across one to many machines).

`lightning` could handle all these with just a parameter twicking - huge benefit.

Reference:
https://huggingface.co/docs/transformers/perf_train_gpu_many

## 3) Data Parallelism for multi-GPU Training

Concept of `data parallelism` and its extension - `distributed data parallelism`, both essential strategies for accelerating machine learning training using multiple computational resources.

**Data parallelism** is a technique where the training data is divided into multiple subsets, and each subset is processed independently across multiple GPUs or computing nodes. This allows for simultaneous computation, significantly reducing the training time. The computed gradients from each subset are then aggregated to update the model parameters. However, in the context of a single machine with multiple GPUs, this method can be limited by the inter-GPU communication speed and the machine's memory capacity.

To overcome the limitations of regular data parallelism in PyTorch, we discussed **distributed data parallelism**, an extension of data parallelism that spans across multiple machines, each with one or more GPUs. In distributed data parallelism, the same model is replicated on each machine, and every machine processes a different subset of the training data. This not only facilitates handling larger datasets and models but also improves the training speed, taking advantage of the collective memory and computational power of multiple machines.

Example:
DistilBert with mixed precision training on 1 GPU - 7 mins with acc 92%.

DistilBert with mixed precision training on 4 GPUs (ddp strategy) - 2.5 mins (x3 faster) with acc 92% (identical).


**Data parallelism** in a nutshell:
Suppose minibatch is too large to fit into GPU memory. *Idea* - split it into smaller batches. Put each microbatch onto a different GPU. Make a copy of the neural network for each GPU.

1) Transfer model, micro batches to GPUs
2) On each GPU - forward pass to compute logits
3) Transfer logits to one GPU
4) On that GPU compute loss, gradients, weight update
5) Update each model copy on each GPU.

Compute forward/backward passes on each GPU in parallel. Each GPU operates independently on the copy of the model. Average gradients, update weights, update the model copies on different GPUs.

**Note** - model operates on smaller batches (gradients are computed on smaller dataset that is averaged). --> Smaller minibatches require smaller learning rates.

**Linear scaling rule**: when the minibatch size is multiplied by k, multiply the learning rate by k.

The learning rate is typically scaled linearly with the batch size. That means if you halve the batch size, you would also halve the learning rate. This is known as "linear scaling rule".

**Distributed data parallelism** - recommended strategy in practice.

Each GPU gets the full batch size. Let `batch_size = 32` each GPU get a different mini-batch but all the mini-batches have the size of 32. In contrast to data parallelism where we split the mini batch into micro batches.

(+) We do not have to adjust the learning rate wrt to the batch size.

1) Transfer model, mini batches to GPUs
2) On each GPU - forward pass to compute logits
3) On each GPU - compute the backward, loss, gradients
4) Communication between GPUs to update the weights.

To be short, fewer transfers steps in Distributed data parallelism which is faster.

### Data Parallel vs Distributed Data Parallel:

Data Parallel
- older implementation (may be deprecated)
- a single process for multiple devices
- 1 host GPU sends data and weights and collects gradients


DistributedDataParallel (*recommended*)
- more modern implementation
- multiple processes for multiple devices
- each device handles its own data, each device performs gradient calc & update
- faster - less data transfers involved
- does not split the batch! (the batch size is per device and per node)

Q. Say you have trained a model with a batch size of 64. Now you use regular data parallelism with 4 GPUs. -> You should use a smaller learning rate (learning rate scaled linearly with the batch size).

Q. Say you have trained a model with a batch size of 64. Now you use distributed data parallelism with 4 GPUs. -> You should use the same learning rate. (It is likely that the same learning rate still works well because distributed data parallelism does not split the minibatches further into microbatches).

## 4) Compiling PyTorch Models

`torch.compile` - a new feature in PyTorch 2.0 that allows you to speed up your PyTorch code by JIT-compiling it into optimized kernels.

`torch.compile` is a fully additive feature, which means that it does not require you to make any changes to your existing code.

As we've seen, to use **torch.compile**, we can simply use the `torch.compile()` function for an exisiting PyTorch model without making any further modifications. The `torch.compile()` will then compile the model into an optimized kernel the first time it is called. Subsequent calls to the function or module will be much faster, as they will be executed directly on the GPU.

The 3 compilation steps:
1) Graph acquisition
- construct the computation graph based on model definition (forward pass)

2) Graph lowering
- represent graph using simpler operations
- *primary purpose* - Convert a PyTorch model into a form that's more amenable to optimization and execution on certain hardware.  --> This transformation is essential for optimization and efficient execution on certain hardware platforms.

3) Graph compilation
- make it run on the hardware (cpu or gpu)

In [ ]:
model = torch.compile(model)
# performance boost just by using one line

Example:
DistilBert with mixed precision training on 1 GPU - 7 mins with acc 92%.

DistilBert with mixed precision training on 1 GPU + `torch.compile` - 3.7 mins (x2 faster) with acc 92% (identical).

**Rule of thumb**: the longer you train and the bigger the model - the more benefits from compilation. The initial compulation introduces some overhead.

We have seen that PyTorch `torch.compile` can speed up the model training.
However, what are some of the **limitations** of torch.compile:

- Not all PyTorch models can be compiled with torch.compile
- The compilation process can be time-consuming (overhead)
- The compiled models may not be as portable as the original PyTorch models.

## 5) Increasing Batch Sizes to Increase Throughput

**Increasing batch sizes** to boost throughput in machine learning model training. A larger batch size can often result in faster model convergence or better end performance.

**Impact**. The batch size, or the number of training samples processed before the model is updated, plays a critical role in the efficiency and effectiveness of model training. By increasing the batch size, we can process more data simultaneously, leading to higher computational efficiency and increased throughput, particularly on hardware like GPUs which excel in parallel processing.

However, in practice, throughput is not always everything, and we have to make sure to strike a careful *balance* between batch size, learning rate, computational resources, and the potential impact on model performance, which are all crucial considerations in machine learning training pipelines.

**Size**. Large batch size are better but only up to a certain point (larger than 1024 not useful). Too small batch sizes can be bad when using *BatchNorm* and hurt the predictive performance.


Q. Why might using a very large batch size in deep learning training sometimes lead to less accurate results? -> It can lead to fewer updates per epoch, which can decrease the amount of implicit regularization.

More smaller updates (smalled batch size) can lead to more small noise, which can sometimes be beneficial. However, often large batch sizes work just as well if we use a learning rate scheduler.

**Warm-starting** large batch-size training with small batch sizes at the start can help with generalization performance.

## Tricks to reduce the memory footprint and speed up training for large models

https://huggingface.co/docs/transformers/v4.18.0/en/performance

https://huggingface.co/docs/transformers/performance

### 6) Gradient Accumulation

- a simple trick to effectively train larger batch size

The **idea** behind gradient accumulation is to instead of calculating the gradients for the whole batch at once to do it in smaller steps.

**How?** - The way we do that is to calculate the gradients iteratively in smaller batches by doing a forward and backward pass through the model and accumulating the gradients in the process. When enough gradients are accumulated we run the model's optimization step.

**Impact** - This way we can easily increase the overall batch size to numbers that would never fit into the GPU's memory. In turn, however, the added forward and backward passes can slow down the training a bit.

**Outcomes** - The memory footprint was dramatically reduced at the cost of being only slightly slower than the vanilla run. In general you would want to max out the GPU usage as much as possible. If we wanted to train with a batch size of 64 we should not use `per_device_train_batch_size=1` and `gradient_accumulation_steps=64` but instead `per_device_train_batch_size=4` and `gradient_accumulation_steps=16` which has the same effective batch size while making better use of the available GPU resources.

Since **gradient accumulation** essentially is identical to having a larger batch size, just as with the larger batch size here you are likely to see a 20-30% speedup due to the optimizer running less often. *Note*: It's important to remember that using gradient accumulation you may end up with a much larger effective batch size, so you may need to adjust the learning rate, its warm up and for very short datasets it'll impact the loss as the training will end up doing less steps than normal.

### 7) Gradient Checkpointing (also known as "activation checkpointing")
- trick to save a little bit more GPU memory


In order to compute the gradients during the backward pass all activations from the forward pass are normally saved. This can create a *big memory overhead*. Alternatively, one could forget all activations during the forward pass and recompute them on demand during the backward pass. This would however add a significant computational overhead and slow down training.

**Gradient checkpointing** strikes a compromise between the two approaches and saves strategically selected activations throughout the computational graph so only a fraction of the activations need to be re-computed for the gradients.

https://medium.com/tensorflow/fitting-larger-networks-into-memory-583e3c758ff9

**Outcome**: We can see that this saved some more memory but at the same time training became a bit slower. A general rule of thumb is that gradient checkpointing slows down training by about 20%.

### 8) Mixed Precision Training

The **idea** of mixed precision training is that no all variables need to be stored in full (32-bit) floating point precision.

If we can reduce the precision the variales and their computations are faster. The main *advantage* comes from saving the activations in half (16-bit) precision. Although the gradients are also computed in half precision they are converted back to full precision for the optimization step so no memory is saved here. Since the model is present on the GPU in both 16-bit and 32-bit precision this can use more GPU memory (1.5x the original model is on the GPU), especially for small batch sizes.

Since some computations are performed in full and some in half precision this approach is also called mixed precision training.

**Outcome**. We can see that with these tweaks we use about half the GPU memory as at the beginning while also being slightly faster

### 9) Low-memory Optimizer (Adam is fat in memory)

The most common optimizer used to train transformer model is Adam or AdamW (Adam with weight decay). Adam achieves good convergence by storing the rolling average of the previous gradients which, however, *adds an additional memory* footprint of the order of the number of model parameters. One remedy to this is to use an alternative optimizer such as Adafactor.

### Adafactor

Instead of keeping the rolling average for each element in the weight matrices Adafactor only stores aggregated information (row- and column-wise sums of the rolling averages) which reduces the footprint considerably. One downside of Adafactor is that in some instances convergence can be slower than Adam's (also the convergence of Adafactor can be worse than Adam) so some experimentation is advised here.

**Impact**. We can see that this saves a few more GB on the GPU.

### 8-bit Adam

Instead of aggregating optimizer states like Adafactor, 8-bit Adam keeps the full state and quantizes it. **Quantization** means that it stores the state with lower precision and dequantizes it only for the optimization. This is similar to the idea behind FP16 training where using variables with lower precision saves memory.

**Impact**. We can see that we get a similar memory improvement as with Adafactor while keeping the full rolling average of the gradients.

We need to install the 8-bit optimizer and then pass it as a custom optimizer to the Trainer (in huggingface library).

### 10) Multi-GPU Training

If your model fits on a single GPU scaling to many GPUs can be achieved fairly easily with **data parallelism**.

The *idea* is very similar to gradient accumulation with the distinction that instead of running the forward and backward passes during the accumulation in sequence on a single machine they are performed in parallel on multiple machines. So each GPU gets a small batch, runs the forward and backward passes and then the gradients from all machines are aggregated and the model is optimized.

If the model does not fit on a single GPU with all the mentioned tricks there are still more methods we can apply although life starts to get a bit more complicated. This usually involves some form of **pipeline or tensor parallelism** where the model itself is distributed across several GPUs.

One can also make use of DeepSpeed which implements some of these parallelism strategies along with some more optimization to reduce the memory footprint such as partitioning the optimizer states.

https://huggingface.co/docs/transformers/v4.18.0/en/parallelism

### 11) DP vs DDP

DistributedDataParallel (DDP) is typically faster than DataParallel (DP), but it is not always the case:

- while DP is python threads-based, DDP is multiprocess-based - and as such it has no python threads limitations, such as GIL
- on the other hand a slow inter-connectivity between the GPU cards could lead to an actual slower outcome with DDP

DDP:
- At the start time the main process replicates the model once from gpu 0 to the rest of gpus
- Then for each batch:
    - each gpu consumes each own mini-batch of data directly
    - during backward, once the local gradients are ready, they are then averaged across all processes

DP:
For each batch:
- gpu 0 reads the batch of data and then sends a mini-batch to each gpu
- replicates the up-to-date model from gpu 0 to each gpu
- runs forward and sends output from each gpu to gpu 0, computes loss
- scatters loss from gpu 0 to all gpus, runs backward
- sends gradients from each gpu to gpu 0 and averages those

**Summary**: The only communication DDP performs per batch is sending gradients, whereas DP does 5 different data exchanges per batch
- Main differences in the inter-GPU communication overhead between the two modes.
- Under DP gpu 0 performs a lot more work than the rest of the gpus, thus resulting in under-utilization of gpus.
- You can use DDP across multiple machines, but this is not the case with DP.



### 12) DataLoader

One of the important requirements to reach great training speed is the ability to feed the GPU at the maximum speed it can handle. By default everything happens in the main process and it might not be able to read the data from disk fast enough, and thus create a **bottleneck**, leading to GPU under-utilization.

- DataLoader(pin_memory=True, ...) which ensures that the data gets preloaded into the pinned memory on CPU and typically leads to much faster transfers from CPU to GPU memory.
- DataLoader(num_workers=4, ...) - spawn several workers to pre-load data faster.

## Summary Table

Each method can improve speed or memory usage which is summarized in the table below:

| Method | Speed | Memory |
| -------- | -------- | -------- |
| Gradient accumulation    | No    | Yes    |
| Gradient checkpointing    | No    | Yes    |
| Mixed precision training    | Yes    | (No)    |
| Batch size    | Yes    | Yes    |
| Optimizer choice    | Yes    | Yes    |
| DataLoader    | Yes    | No    |
| DeepSpeed Zero    | No    | Yes    |
| Compile    | Yes    | No    |

DeepSpeed decision tree:
- Model fits onto a single GPU and you have enough space to fit a small batch size - you don't need to use Deepspeed as it'll only slow things down in this use case.
- Model doesn't fit onto a single GPU or you can't fit a small batch - use DeepSpeed